# Building and Exploring the dataset

In [3]:
df_books.head()

,Unnamed: 0,context,tags,book_id,section_id
0,0,\r\n\r\nIt is a truth universally acknowledged...,100\1.txt,100,1
1,1,\r\n\r\nThe next day opened a new scene at Lon...,100\10.txt,100,10
2,2,\r\n\r\nThe discussion of Mr. Collins's offer ...,100\11.txt,100,11
3,3,\r\n\r\nElizabeth was sitting with her mother ...,100\12.txt,100,12
4,4,\r\n\r\nAfter a week spent in professions of l...,100\13.txt,100,13


In [4]:
df_questions.head()

,question_id,question,answer,book_id
0,0,1. Where is Alice when she sees the White\r\r\...,On a riverbank.,77
1,1,2. Who is Alice with before her adventures\r\r...,Her sister.,77
2,2,3. What is the rabbit looking at when\r\r\nAli...,His watch.,77
3,3,4. What does Alice pick up while falling\r\r\n...,An empty jar.,77
4,4,5. Alice does NOT think she might end up\r\r\n...,America.,77


In [38]:
import pandas as pd 
import numpy as np
#word manipulation
from nltk.tokenize import word_tokenize  

df_crosswalk = pd.read_csv('C:\\Users\\Jennifer\\Documents\\Berkeley\\W266\\Final Project\\Final-dataset\\midpoint_crosswalk.csv')
df_books = pd.read_csv('C:\\Users\\Jennifer\\Documents\\Berkeley\\W266\\Final Project\\Final-dataset\\midpoint_context_fv.csv', '\t',encoding='utf-8')
df_questions = pd.read_csv('C:\\Users\\Jennifer\\Documents\\Berkeley\\W266\\Final Project\\Final-dataset\\midpoint_questions_fv.csv', '\t',encoding='utf-8')

df_merge = pd.merge(df_crosswalk, df_questions, on=['book_id','question_id'], how='outer')
df_merge2 = pd.merge(df_merge, df_books, on=['book_id','section_id'], how='outer')

df_merge2["train_text"] = df_merge2["question"].map(str) +str(' ') +  df_merge2["context"]

In [39]:
df_model = df_merge2[["train_text", "answer"]]

In [40]:
df_model.head()

,train_text,answer
0,2. Who is Marmee? CHAPTER ONE\r\n\r\nPLAYING P...,the mother
1,3. Marmee\r\r\nreceives a letter from her husb...,serving in the army
2,4. What does each girl find under her\r\r\npil...,a book
3,5. What is the name of the family's long\r\r\n...,Hannah
4,6. What is the last name of the family\r\r\nth...,Hummel


In [41]:
df_model.iloc[0,0]

u'2. Who is Marmee? CHAPTER ONE\r\n\r\nPLAYING PILGRIMS\r\n\r\n"Christmas won\'t be Christmas without any presents," grumbled Jo, lying\r\non the rug.\r\n\r\n"It\'s so dreadful to be poor!" sighed Meg, looking down at her old\r\ndress.\r\n\r\n"I don\'t think it\'s fair for some girls to have plenty of pretty\r\nthings, and other girls nothing at all," added little Amy, with an\r\ninjured sniff.\r\n\r\n"We\'ve got Father and Mother, and each other," said Beth contentedly\r\nfrom her corner.\r\n\r\nThe four young faces on which the firelight shone brightened at the\r\ncheerful words, but darkened again as Jo said sadly, "We haven\'t got\r\nFather, and shall not have him for a long time." She didn\'t say\r\n"perhaps never," but each silently added it, thinking of Father far\r\naway, where the fighting was.\r\n\r\nNobody spoke for a minute; then Meg said in an altered tone, "You know\r\nthe reason Mother proposed not having any presents this Christmas was\r\nbecause it is going to be a har

In [42]:
df_model.iloc[0,1]

u'the mother'

In [43]:
df_train_data = df_model.iloc[0:4000,0]
df_train_labels = df_model.iloc[0:4000,1]

In [44]:
df_train_data.shape

(4000L,)

In [45]:
df_train_data.head()

0    2. Who is Marmee? CHAPTER ONE\r\n\r\nPLAYING P...
1    3. Marmee\r\r\nreceives a letter from her husb...
2    4. What does each girl find under her\r\r\npil...
3    5. What is the name of the family's long\r\r\n...
4    6. What is the last name of the family\r\r\nth...
Name: train_text, dtype: object

In [46]:
df_train_labels.head()

0             the mother
1    serving in the army
2                 a book
3                 Hannah
4                 Hummel
Name: answer, dtype: object

In [47]:
df_test_data = df_model.iloc[4000:,0]
df_test_labels = df_model.iloc[4000:,1]

In [48]:
df_test_data.shape

(1277L,)

In [56]:
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
##################################################
#create n grams
#default CountVectorizer options 
#singleGram 
vectorizer = CountVectorizer() #decode_error="replace")#strip_accents= 'unicode', encoding='utf-8',
train_vec = vectorizer.fit_transform(df_train_data)#.values.astype('U') #str), 'U'
#train_vec.shape
#train_array = train_vec.toarray()
#print train_array [1:20,]
test_vec = vectorizer.transform(df_test_data)
#print test_vec.shape

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
from sklearn.linear_model import LogisticRegression
##################################################
#Build LR model
#fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization.
#LR Model
regularization_c = {'C': [0.0001, 0.001, 0.01, 0.1 ]} #,0.25, 0.5, 0.75, 0.9, 0.99,1]} #5,10,100]} #added 0.25 and 0.75 after seeing that 0.5 performed well, dropped above 1 at that point too
LR_model = LogisticRegression(penalty="l2")
grid = GridSearchCV(estimator=LR_model, param_grid=regularization_c, scoring = "f1_weighted")
grid.fit(train_vec, train_labels)
#summarize the results of the grid search
print "Logistic Regression best f1 score for training data: " + str(grid.best_score_)
print "Logistic Regression best regularization parameter: "+ str(grid.best_estimator_.C)


# Trying different encodings - to see what piece is breaking the tokenizer

In [68]:
df_questions2 = pd.read_csv('C:\\Users\\Jennifer\\Documents\\Berkeley\\W266\\Final Project\\Final-dataset\\midpoint_questions.csv', '\t',encoding='utf-8')

In [70]:
df_questions2.head()

,question_id,question,answer,book_id
0,0,1. Where is Alice when she sees the White\r\r\...,On a riverbank.,77
1,1,2. Who is Alice with before her adventures\r\r...,Her sister.,77
2,2,3. What is the rabbit looking at when\r\r\nAli...,His watch.,77
3,3,4. What does Alice pick up while falling\r\r\n...,An empty jar.,77
4,4,5. Alice does NOT think she might end up\r\r\n...,America.,77


In [71]:
df_questions3 = df_questions2.iloc[:,1]

In [73]:
vectorizer = CountVectorizer()
train_vec = vectorizer.fit_transform(df_questions3).toarray()
#this runs....so the merging of this & th content is the problem; later switche context_builder.py to utf-8, still no luck

In [63]:
df_crosswalk = pd.read_csv('C:\\Users\\Jennifer\\Documents\\Berkeley\\W266\\Final Project\\Final-dataset\\midpoint_crosswalk.csv')

In [64]:
df_books = pd.read_csv('C:\\Users\\Jennifer\\Documents\\Berkeley\\W266\\Final Project\\Final-dataset\\midpoint_context.csv', '\t')

In [ ]:
https://stackoverflow.com/questions/39303912/tfidfvectorizer-in-scikit-learn-valueerror-np-nan-is-an-invalid-document

In [34]:
from nltk.tokenize import word_tokenize 
txt = nltk.word_tokenize (df_test_labels)
txt = nltk.Text(txt)
txt.count("the")

TypeError: expected string or buffer

In [35]:
txt = df_test_data.apply(word_tokenize)
txt.head()

TypeError: expected string or buffer